In [20]:
'''%pip install --disable-pip-version-check \
        torch==1.13.1 \
        torchdata==0.5.1 --quiet

%pip install \
        transformers==4.27.2 \
        datasets==2.11.0 --quiet'''

'%pip install --disable-pip-version-check         torch==1.13.1         torchdata==0.5.1 --quiet\n\n%pip install         transformers==4.27.2         datasets==2.11.0 --quiet'

In [21]:
'''%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet'''

'%pip install --upgrade pip\n%pip install --disable-pip-version-check     torch==1.13.1     torchdata==0.5.1 --quiet\n\n%pip install     transformers==4.27.2     datasets==2.11.0  --quiet'

In [1]:
from config import reddit_client_id, reddit_client_secret, reddit_user_agent

In [10]:
import praw
# Initialize the Reddit API wrapper
reddit = praw.Reddit(client_id=reddit_client_id,
                     client_secret=reddit_client_secret,
                     user_agent=reddit_user_agent)

# Replace 'python' with the name of the subreddit you want to pull posts from
subreddit = reddit.subreddit('CryptoCurrency')

for submission in reddit.subreddit("CryptoCurrency").hot(limit=10):
    print(submission.title)

#TODO: extract headings and top level comments for top n threads (hot n?)
#TODO: store this where? Patch as json to some store - blob?

NameError: name 'reddit_client_id' is not defined

In [ ]:
# compile the dataset
submissions_dataset = []
comments_dataset = []

for submission in reddit.subreddit("CryptoCurrency").hot(limit=10):
    submissions_dataset.append([submission.title, submission.date])
    for top_level_comment in submission.comments:
        if isinstance(top_level_comment, MoreComments):
            dataset.append(top_level_comment.body)

Note: # From qualitative analysis: a comment doesn't often have multiple coins+strong sentiment associated with each one, usually it's one coin mention and discussion, it may mention other coins but the comment itself will center around one of them and only have strong sentiment associated with that one coin. Comments like "BTC is amazing, Etherium sucks, Doge is the best" are rare. 

In [ ]:
#TODO: what are we extracting? Coin mentions, sentiment??


In [ ]:
# TODO: BTC/Bitcoin mentions - need to map name/abbreviation to the same coin
# Can potentially be done by the LLM
# If not, can scrape the coinwire page and write a function to map

In [2]:
#TODO: load the base model using tensorflow

# Loading the base model, the tokenizer and the config
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [4]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [5]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [7]:
# Tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, batched = True)

In [ ]:
#TODO: write a prompt

prompt = ''''''